<a href="https://colab.research.google.com/github/Bambika69/Alfa_Csapat_Grafikus/blob/main/Prep%26FineTune/TRANSLATION_INSTRUCTION_EN_TO_HU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.5 MB/s eta 0:00:00


In [2]:
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [3]:
!git clone https://github.com/tloen/alpaca-lora.git

Cloning into 'alpaca-lora'...
remote: Enumerating objects: 607, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 607 (delta 28), reused 33 (delta 19), pack-reused 556
Receiving objects: 100% (607/607), 27.78 MiB | 6.81 MiB/s, done.
Resolving deltas: 100% (360/360), done.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-hu")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-hu")

In [ ]:
!pip install pandas

In [7]:
import pandas as pd

def reduce_json_size(input_file, output_file):
    # Read the JSON file into a pandas DataFrame
    df = pd.read_json(input_file)

    # Calculate the number of elements to keep
    reduced_size = len(df) // 4

    # Keep only the desired elements
    reduced_df = df[:reduced_size]

    # Write the reduced data to a JSON file
    reduced_df.to_json(output_file, orient='records', indent=4)

# Usage example
reduce_json_size("/content/alpaca-lora/alpaca_data.json", "/content/alpaca-lora/reduced.json")

In [14]:
alpaca_reduced_data = json.load(open("/content/alpaca-lora/reduced.json", "r"))
alpaca_original_data = json.load(open("/content/alpaca-lora/alpaca_data.json", "r"))

In [17]:
print(alpaca_original_data[-1])
print(alpaca_reduced_data[-1])

{'instruction': 'Analyze the given legal document and explain the key points.', 'input': 'The following is an excerpt from a contract between two parties, labeled "Company A" and "Company B": \n\n"Company A agrees to provide reasonable assistance to Company B in ensuring the accuracy of the financial statements it provides. This includes allowing Company A reasonable access to personnel and other documents which may be necessary for Company B’s review. Company B agrees to maintain the document provided by Company A in confidence, and will not disclose the information to any third parties without Company A’s explicit permission."', 'output': 'This legal document states that Company A has agreed to provide reasonable assistance to Company B in ensuring the accuracy of the financial statements. Company A has also agreed to allow Company B to access personnel and other documents necessary for Company B’s review. Company B, in turn, has accepted responsibility to maintain the confidentialit

In [18]:
translator = pipeline("translation_EN_to_HU", model=model, tokenizer=tokenizer, max_length=1024)

In [19]:
alpaca_hun_data = []
for conversation in tqdm(alpaca_reduced_data):
    conv_hu = dict()
    
    try:
        if any(conversation["input"]):
    
            conv_inst = conversation["instruction"]
            conv_input = conversation["input"]
            conv_output = conversation["output"]

            
    
            conv_hu["instruction"] = translator(conv_inst)[0]["translation_text"]
            conv_hu["input"] = translator(conv_input)[0]["translation_text"]
            conv_hu["output"] = translator(conv_output)[0]["translation_text"]
        else:
            conv_inst = conversation["instruction"]
            conv_output = conversation["output"]

            conv_hu["instruction"] = translator(conv_inst)[0]["translation_text"]
            conv_hu["input"] = ""
            conv_hu["output"] = translator(conv_output)[0]["translation_text"]
    except Exception as e:
         print(e)
    alpaca_hun_data.append(conv_hu)

100%|██████████| 13000/13000 [8:35:58<00:00,  2.38s/it]


In [20]:
verified_alpaca_data = []
i = 0
for row in tqdm(alpaca_hun_data):
    if row.get("instruction") and row.get("output"):
        verified_alpaca_data.append(row)
    else:
        print(row)
        i += 1
print(i)

100%|██████████| 13000/13000 [00:00<00:00, 1070353.58it/s]

{'instruction': 'Az alábbi mondat fordítása japánul', 'input': 'Ő egy kiváló tanár.', 'output': ''}
{'instruction': 'Válassz egy hangulatjelet, amely a legjobban leírja a következő hangulatot.', 'input': 'Izgatott', 'output': ''}
{'instruction': 'A következő Unicode-szöveg konvertálása ASCII-be', 'input': '"U0001F608"', 'output': ''}
3


In [21]:
len(verified_alpaca_data)
json.dump(verified_alpaca_data, open("/content/alpaca-lora/alpaca_data_hu_verified.json", "w"))